In [ ]:
import osmnx as ox
import pandas as pd

In [ ]:
coord_origen = (40.405551, -3.700684 ) #ETSIDI
coord_destino = (40.408800, -3.701271 )#Plaza Lavapiés
mapa_nodos = ox.graph_from_point (coord_origen, dist=7000, dist_type ='bbox' , network_type = 'drive')
fig, ax = ox.plot_graph(mapa_nodos) 
ox.speed.add_edge_speeds(mapa_nodos , hwy_speeds = 50 , fallback = None , precision = 1 )
ox.speed.add_edge_travel_times( mapa_nodos , precision = 1 )

In [ ]:
#---------------------NODOS ORIGEN Y DESTINO SEGÚN COORDENADAS----------------------------------#
nodo_origen = ox.get_nearest_node(mapa_nodos, coord_origen, method='haversine', return_dist=False)
nodo_destino = ox.get_nearest_node(mapa_nodos, coord_destino, method='haversine', return_dist=False)
nodo_origen,nodo_destino

In [ ]:
#GRAFICAR A GEODATAFRAME
nodos, caminos = ox.graph_to_gdfs(mapa_nodos) #Función de OSMnx que convierte el multidigraph en geodataframes de NODOS y LÍNEAS
pd.set_option("max_columns", None) #Showing only two columns
pd.set_option("max_rows", None)
nodos.head() #GeoDataFrames de los nodos

In [ ]:
caminos.head()

In [ ]:
#--------------------------NODOS DE LA RUTA---------------------------------------#
ruta = ox.shortest_path(G=mapa_nodos, orig=nodo_origen, dest=nodo_destino, weight='length')
print(ruta) #Obtenemos la lista de nodos que se encuentran en el camino más corto.
len(ruta)

In [ ]:
#--------------------------INFORMACIÓN CALLES DE LA RUTA ---------------------------------------#
lista_caminos =[]
for x in range(1,len(ruta)):
    lista_caminos.append(mapa_nodos[ruta[x-1]][ruta[x]])
len(lista_caminos)

In [ ]:
#--------------------------NOMBRE DE LAS CALLES---------------------------------------#
nombre_calles =[]
for x in range(0,len(lista_caminos)):
    nombre_calles.append(lista_caminos[x][0]['name'])
len(nombre_calles)

In [ ]:
#--------------------------NODOS INICIALES DE LAS CALLES---------------------------------------#
nodo1_calle =[]
for x in range(0,len(ruta)-1):
    nodo1_calle.append(ruta[x])
len(nodo1_calle)

In [ ]:
#--------------------------NODOS FINALES DE LAS CALLES---------------------------------------#
nodo2_calle =[]
for x in range(1,len(ruta)):
    nodo2_calle.append(ruta[x])
len(nodo2_calle)


In [ ]:
#--------------------------COORDENADAS TOTALES EN UNA LISTA---------------------------------------#
lista_coords_nodos =[]
for x in range(0,len(ruta)):
    lista_coords_nodos.append(mapa_nodos.nodes[ruta[x]]['y'])
    lista_coords_nodos.append(mapa_nodos.nodes[ruta[x]]['x'])
len(lista_coords_nodos)

In [ ]:
#--------------------------COORDENADAS FINALES DE LA RUTA---------------------------------------#
coords_nodos = []
for i in range(0, len(lista_coords_nodos), 2):
    coords_nodos.append(lista_coords_nodos[i:i+2])
len(coords_nodos)

In [ ]:
#--------------------------COORDENADAS INICIALES DE LAS CALLES---------------------------------------#
coords1_calle =[]
for x in range(0,len(ruta)-1): #Elimino la ultima coordenada para tener longitud 30
    coords1_calle.append(coords_nodos[x])
len(coords1_calle) 

In [ ]:
#-------------------------COORDENADAS FINALES DE LAS CALLES--------------------------------------#
coords2_calle =[]
for x in range(1,len(ruta)):
    coords2_calle.append(coords_nodos[x])
len(coords2_calle)

In [ ]:
#----------------------------TRAZADO DE LA RUTA-------------------------------------------#
caminos.plot(ax=ax, linewidth=1, edgecolor='dimgray') #Líneas
fig2, ax = ox.plot_graph_route(mapa_nodos, ruta)

In [ ]:
#-------------------------INFORMACIÓN POR NODO DE LA RUTA----------------------------------------#
ruta_info=ox.utils_graph.get_route_edge_attributes(mapa_nodos, ruta, attribute=None, minimize_key='length', retrieve_default=None)
ruta_info

In [ ]:
#----------------------------VELOCIDAD POR CALLE EN KPH-------------------------------------------#
calles_speed =[]
for x in range(0,len(ruta_info)):
    calles_speed.append(ruta_info[x]['speed_kph'])
len(calles_speed)

In [ ]:
#----------------------------TIEMPO POR CALLE EN SEGS-------------------------------------------#
calles_tiempo =[]
for x in range(0,len(ruta_info)):
   calles_tiempo.append(ruta_info[x]['travel_time'])
len(calles_tiempo)

In [ ]:
#----------------------------HORAS INTERMEDIAS DE LA RUTA-------------------------------------------#
from datetime import datetime, date, time, timedelta
from datetime import date
from datetime import datetime
hora_salida = datetime(2021, 2, 28, 10, 15, 00) #y, m, d, h, min, s
horas_intermedias = hora_salida
horas_trayecto = []
for x in range(0,len(ruta_info)):
   horas_trayecto.append(horas_intermedias + timedelta(seconds=ruta_info[x]['travel_time']))
   horas_intermedias += timedelta(seconds=ruta_info[x]['travel_time'])

In [ ]:
#----------------------------------------DATAFRAME-------------------------------------------#
df = pd.DataFrame(nombre_calles,columns=['Dirección'])
df['km/h'] = calles_speed
df['Tiempo'] = calles_tiempo
df['Nodo Origen'] = nodo1_calle
df['Origen'] = coords1_calle
df['NodDestino'] = nodo2_calle
df['Destino'] = coords2_calle
df['Ruta'] = horas_trayecto



In [ ]:
print ('Fecha y hora de inicio de la ruta', hora_salida)
df.iloc[0:len(ruta)] # Primeras cinco filas